

# Advanced SQL: DVD Rental Analysis

This project demonstrates advanced SQL techniques applied to a sample DVD rental database (based on the Sakila dataset, representing a fictional company "Rentio"). Each section presents a business question relevant to Rentio and the SQL query used to answer it, highlighting specific SQL features.

**Database Context:**

The analysis uses a star schema with tables like `fact_rental`, `dim_film`, `dim_customer`, `dim_store`, `dim_category`, etc., containing information about rentals, films, customers, stores, and categories.

---

## Business Questions & SQL Queries

**1. Analyze Genre Performance by Store (Summer 2005)**

* **Business Question:** What was the total rental revenue generated by the 'Travel', 'Family', and 'Children' film categories in each store during June and July of 2005?
* **Context:** Understanding the performance of family-oriented genres during the summer months across different store locations to inform inventory and marketing decisions.
* **SQL Practice:** `JOIN`, `WHERE` with `IN` and `BETWEEN`, `SUM()`, `GROUP BY`, `ORDER BY`.

```sql
SELECT
    fr.store_id,
    dc.name AS category_name,
    SUM(fr.amount) AS total_amount
FROM
    fact_rental fr
    INNER JOIN dim_category dc ON fr.category_id = dc.category_id
WHERE
    dc.name IN ('Family', 'Travel', 'Children')
    AND fr.rental_date BETWEEN '2005-06-01' AND '2005-08-01' -- Note: BETWEEN includes endpoints, so this covers June and July.
GROUP BY
    fr.store_id,
    dc.name
ORDER BY
    fr.store_id,
    dc.name;
```

---

**2. Calculate Average Film Count per Category**

* **Business Question:** What is the average number of distinct films offered across all categories? Provide the precise average, as well as the average rounded down and rounded up.
* **Context:** Assessing the overall diversity and balance of the film catalog across different genres.
* **SQL Practice:** Multiple `CTEs`, `DISTINCT`, `COUNT()`, `AVG()`, `FLOOR()`, `CEIL()`.

```sql
WITH film_category AS (
    -- Get distinct film-category pairs
    SELECT DISTINCT
        category_id,
        film_id
    FROM
        fact_rental
),
film_category_count AS (
    -- Count distinct films per category
    SELECT
        category_id,
        COUNT(film_id) AS films
    FROM
        film_category
    GROUP BY
        category_id
),
films_average_by_category AS (
    -- Calculate the average count across categories
    SELECT
        AVG(films) AS average_by_category
    FROM
        film_category_count
)
-- Final calculation and presentation
SELECT
    average_by_category AS average,
    FLOOR(average_by_category) AS average_down,
    CEIL(average_by_category) AS average_up
FROM
    films_average_by_category;
```

---

**3. Identify Categories with Above-Average Film Counts**

* **Business Question:** Which film categories feature a number of distinct films greater than the rounded-up average number of films per category?
* **Context:** Highlighting categories with a significantly larger selection than the typical category, potentially indicating areas of focus or overstocking.
* **SQL Practice:** `CTEs`, `Subquery` in `WHERE` clause, `JOIN`, `COUNT()`, `AVG()`, `CEIL()`, `GROUP BY`.

```sql
WITH film_category AS (
    -- Get distinct film-category pairs with names
    SELECT DISTINCT
        dc.category_id,
        dc.name AS category,
        fr.film_id
    FROM
        fact_rental fr
        INNER JOIN dim_category dc ON dc.category_id = fr.category_id
),
film_category_count AS (
    -- Count distinct films per category
    SELECT
        category,
        category_id,
        COUNT(film_id) AS films
    FROM
        film_category
    GROUP BY
        category,
        category_id
)
-- Select categories with film count above the rounded-up average
SELECT
    category,
    films
FROM
    film_category_count
WHERE
    films > (
        -- Subquery to calculate the rounded-up average
        SELECT
            CEIL(AVG(films))
        FROM
            film_category_count
    )
ORDER BY
    category;
```

---

**4. Categorize Customer Spending Levels (Specific Time Window)**

* **Business Question:** For payments processed on April 30th, 2007, between 3 PM and 4 PM, what was each customer's maximum single transaction amount, and can we classify these amounts as 'low' (0-3), 'mid' (3-6), or 'high' (6+)?
* **Context:** Analyzing customer spending patterns during a specific, potentially high-traffic hour, and segmenting customers based on their peak spending value.
* **SQL Practice:** `CTE`, `WHERE` with `BETWEEN` on `TIMESTAMP`, `MAX()`, `DATE()`, `GROUP BY`, `JOIN`, `CONCAT()`, `UPPER()`, `CASE` statement, `ORDER BY`.

```sql
WITH max_amount_customer AS (
    -- Find max amount per customer within the time window
    SELECT
        customer_id,
        DATE(payment_date) AS payment_date, -- Extract only the date part
        MAX(amount) AS max_amount
    FROM
        fact_rental
    WHERE
        payment_date BETWEEN '2007-04-30 15:00:00' AND '2007-04-30 16:00:00'
    GROUP BY
        customer_id,
        DATE(payment_date)
)
-- Join with customer details and apply spending category
SELECT
    CONCAT(
        UPPER(dc.first_name),
        ' ',
        UPPER(dc.last_name)
    ) AS full_name,
    mac.max_amount,
    mac.payment_date,
    CASE
        WHEN mac.max_amount >= 0 AND mac.max_amount < 3 THEN 'low'
        WHEN mac.max_amount >= 3 AND mac.max_amount < 6 THEN 'mid'
        WHEN mac.max_amount >= 6 THEN 'high'
        -- Optional: ELSE 'undefined' if amount could be negative or NULL
    END AS value_rate -- Renamed from max_amount in original to avoid confusion
FROM
    max_amount_customer mac
    INNER JOIN dim_customer dc ON dc.customer_id = mac.customer_id
ORDER BY
    mac.max_amount DESC,
    full_name ASC;
```

---

**5. Create a Customer Spending Pivot Table by Category**

* **Business Question:** How can we display the total amount spent by each customer, with individual columns representing the total spending within each film category?
* **Context:** Generating a summarized report (pivot table) that allows for easy comparison of customer spending habits across all movie genres.
* **SQL Practice:** `CTE`, `JOIN`, `SUM()`, `GROUP BY`, `CASE` within aggregate (`MAX`) for pivoting.

```sql
WITH customer_category_sum AS (
    -- Calculate total spending per customer per category
    SELECT
        CONCAT(
            dc.first_name,
            ' ',
            dc.last_name
        ) AS full_name,
        dcat.name AS category,
        SUM(fr.amount) AS amount
    FROM
        fact_rental fr
        INNER JOIN dim_customer dc ON dc.customer_id = fr.customer_id
        INNER JOIN dim_category dcat ON dcat.category_id = fr.category_id
    GROUP BY
        full_name,
        category
)
-- Pivot the results using MAX(CASE...)
SELECT
    full_name,
    MAX(CASE WHEN category = 'Action' THEN amount ELSE 0 END) AS "Action",
    MAX(CASE WHEN category = 'Animation' THEN amount ELSE 0 END) AS "Animation",
    MAX(CASE WHEN category = 'Children' THEN amount ELSE 0 END) AS "Children",
    MAX(CASE WHEN category = 'Classics' THEN amount ELSE 0 END) AS "Classics",
    MAX(CASE WHEN category = 'Comedy' THEN amount ELSE 0 END) AS "Comedy",
    MAX(CASE WHEN category = 'Documentary' THEN amount ELSE 0 END) AS "Documentary",
    MAX(CASE WHEN category = 'Drama' THEN amount ELSE 0 END) AS "Drama",
    MAX(CASE WHEN category = 'Family' THEN amount ELSE 0 END) AS "Family",
    MAX(CASE WHEN category = 'Foreign' THEN amount ELSE 0 END) AS "Foreign",
    MAX(CASE WHEN category = 'Games' THEN amount ELSE 0 END) AS "Games",
    MAX(CASE WHEN category = 'Horror' THEN amount ELSE 0 END) AS "Horror",
    MAX(CASE WHEN category = 'Music' THEN amount ELSE 0 END) AS "Music",
    MAX(CASE WHEN category = 'New' THEN amount ELSE 0 END) AS "New",
    MAX(CASE WHEN category = 'Sci-Fi' THEN amount ELSE 0 END) AS "Sci-Fi",
    MAX(CASE WHEN category = 'Sports' THEN amount ELSE 0 END) AS "Sports",
    MAX(CASE WHEN category = 'Travel' THEN amount ELSE 0 END) AS "Travel"
FROM
    customer_category_sum
GROUP BY
    full_name
ORDER BY
    full_name;
```

---

**6. Assess On-Time vs. Late Returns (Specific Time Window)**

* **Business Question:** For rentals paid for on April 30th, 2007, between 3 PM and 4 PM, identify which rentals were returned 'On time' and which were 'Late' based on the film's allowed rental duration.
* **Context:** Evaluating customer return punctuality during a specific timeframe to understand compliance and potential impacts on inventory availability.
* **SQL Practice:** `JOIN`, `WHERE` with `BETWEEN`, `CASE` statement, Date/Time functions (`TIMEDIFF`, `EXTRACT`), Unit Conversion (days to hours).

```sql
SELECT
    dc.customer_id,
    CASE
        -- Compare time difference in hours to allowed duration in hours
        WHEN (EXTRACT(HOUR FROM TIMEDIFF(fr.return_date, fr.rental_date))
              + EXTRACT(DAY FROM TIMEDIFF(fr.return_date, fr.rental_date)) * 24) -- Crude way to get total hours if TIMEDIFF only gives HH:MM:SS part of days
             > df.rental_duration * 24 THEN 'Late'
        -- Add check for return_date IS NULL if needed
        ELSE 'On time'
    END AS delivery_status -- Renamed from 'delivery'
FROM
    fact_rental fr
    INNER JOIN dim_customer dc ON dc.customer_id = fr.customer_id
    INNER JOIN dim_film df ON df.film_id = fr.film_id
WHERE
    fr.payment_date BETWEEN '2007-04-30 15:00:00' AND '2007-04-30 16:00:00'
    -- Consider adding AND fr.return_date IS NOT NULL if only completed rentals matter
ORDER BY
    dc.customer_id;
```
*Note: The accuracy of the `TIMEDIFF` hour extraction for periods longer than 24 hours might depend on the specific SQL dialect. A more robust method might involve calculating the total difference in seconds and dividing.*

---

**7. Generate Staff Initials**

* **Business Question:** How can we generate two-letter initials for each staff member based on their first and last names?
* **Context:** Creating concise identifiers for staff members, useful for reports, system logs, or display purposes where space is limited.
* **SQL Practice:** String functions (`SUBSTRING`, `CONCAT`).

```sql
SELECT
    CONCAT(
        SUBSTRING(first_name, 1, 1),
        SUBSTRING(last_name, 1, 1)
    ) AS initials
FROM
    dim_staff;
```

---

**8. Identify Top Earning Movie per Rating Category**

* **Business Question:** Which movie generated the most rental revenue within each film rating category (e.g., G, PG, R)?
* **Context:** Determining the highest-performing titles based on revenue within specific audience rating segments.
* **SQL Practice:** `CTEs`, `JOIN`, `SUM()`, `GROUP BY`, Window Function (`RANK() OVER (PARTITION BY ... ORDER BY ...)`).

```sql
WITH movies_amount_rating AS (
    -- Calculate total revenue per movie
    SELECT
        df.title,
        df.rating,
        SUM(fr.amount) AS amount
    FROM
        fact_rental fr
        INNER JOIN dim_film df ON df.film_id = fr.film_id
    GROUP BY
        df.title,
        df.rating
),
movies_ranking AS (
    -- Rank movies within each rating based on amount
    SELECT
        title,
        rating,
        amount,
        RANK() OVER (
            PARTITION BY rating
            ORDER BY amount DESC
        ) AS rank_movies
    FROM
        movies_amount_rating
)
-- Select only the top-ranked movie(s) per rating
SELECT
    title,
    rating,
    amount
FROM
    movies_ranking
WHERE
    rank_movies = 1;
```

---

**9. Rank Actors by Number of Films Rented**

* **Business Question:** Who are the top 10 actors based on the count of distinct films they starred in that have been rented out?
* **Context:** Identifying the most prolific actors whose films are present in the rental history, potentially indicating actor popularity or extensive filmography available for rent.
* **SQL Practice:** `CTEs`, `JOIN` (including bridge table), `DISTINCT`, `COUNT(*)`, `GROUP BY`, Window Function (`ROW_NUMBER() OVER (ORDER BY ...)`).

```sql
WITH actor_film AS (
    -- Get distinct actor-film combinations present in rentals
    SELECT DISTINCT
        da.actor_id,
        da.first_name,
        da.last_name,
        df.film_id
    FROM
        dim_actor da
        INNER JOIN bridge_actor ba ON ba.actor_id = da.actor_id
        INNER JOIN fact_rental fr ON fr.rental_id = ba.rental_id
        INNER JOIN dim_film df ON df.film_id = fr.film_id
),
actor_film_count AS (
    -- Count distinct films per actor
    SELECT
        actor_id,
        first_name,
        last_name,
        COUNT(film_id) AS films -- Changed from COUNT(*) to be explicit
    FROM
        actor_film
    GROUP BY
        actor_id, first_name, last_name
),
actor_rank_calc AS ( -- Renamed from actor_rank to avoid confusion with column name
    -- Assign a rank based on film count
    SELECT
        first_name,
        last_name,
        films,
        ROW_NUMBER() OVER (
            ORDER BY films DESC, first_name ASC, last_name ASC -- Added names for tie-breaking
        ) AS actor_rank
    FROM
        actor_film_count
)
-- Select the top 10 ranked actors
SELECT
    first_name,
    last_name,
    films,
    actor_rank
FROM
    actor_rank_calc
WHERE
    actor_rank <= 10
ORDER BY
    actor_rank ASC;

```

---

**10. Analyze Monthly Spending Change (Previous Month)**

* **Business Question:** For a specific customer (ID 388), what was their total rental spending each month, how much did they spend the *previous* month, and what was the difference in spending compared to the previous month?
* **Context:** Tracking month-over-month spending changes for an individual customer to understand their spending patterns and identify trends or anomalies.
* **SQL Practice:** `CTE`, `WHERE`, Date function (`EXTRACT`), `SUM()`, `GROUP BY`, Window Function (`LAG() OVER (ORDER BY ...)`).

```sql
WITH total_payment_amounts_sum AS (
    -- Calculate total spending per month for customer 388
    SELECT
        EXTRACT(MONTH FROM payment_date) AS month,
        EXTRACT(YEAR FROM payment_date) AS year, -- Added year for proper ordering across years
        SUM(amount) AS monthly_amount -- Renamed from amount
    FROM
        fact_rental
    WHERE
        customer_id = 388
        AND payment_date IS NOT NULL
    GROUP BY
        year, month -- Group by year and month
)
SELECT
    year,
    month,
    monthly_amount,
    -- Get previous month's amount using LAG
    LAG(monthly_amount, 1, 0) OVER ( -- Added default value 0 for first month
        ORDER BY year, month -- Order by year then month
    ) AS previous_month_amount,
    -- Calculate the difference (current - previous)
    monthly_amount - LAG(monthly_amount, 1, 0) OVER (
        ORDER BY year, month
    ) AS difference_from_previous -- Renamed from 'difference'
FROM
    total_payment_amounts_sum
ORDER BY
    year, month; -- Ensure final output is chronological
```

---

**11. Analyze Monthly Spending Change (Next Month)**

* **Business Question:** For a specific customer (ID 388), what was their total rental spending each month, how much did they spend the *next* month, and what was the spending difference between the next month and the current month?
* **Context:** Comparing a customer's current monthly spending to their spending in the subsequent month, useful for forecasting or understanding spending momentum.
* **SQL Practice:** `CTE`, `WHERE`, Date function (`EXTRACT`), `SUM()`, `GROUP BY`, Window Function (`LEAD() OVER (ORDER BY ...)`).

```sql
WITH total_payment_amounts_sum AS (
    -- Calculate total spending per month for customer 388
    SELECT
        EXTRACT(MONTH FROM payment_date) AS month,
        EXTRACT(YEAR FROM payment_date) AS year, -- Added year for proper ordering
        SUM(amount) AS monthly_amount -- Renamed from amount
    FROM
        fact_rental
    WHERE
        customer_id = 388
        AND payment_date IS NOT NULL
    GROUP BY
        year, month -- Group by year and month
)
SELECT
    year,
    month,
    monthly_amount,
    -- Get next month's amount using LEAD
    LEAD(monthly_amount, 1, 0) OVER ( -- Added default value 0 for last month
        ORDER BY year, month -- Order by year then month
    ) AS next_month_amount,
    -- Calculate the difference (next - current)
    LEAD(monthly_amount, 1, 0) OVER (
        ORDER BY year, month
    ) - monthly_amount AS difference_to_next -- Renamed from 'difference'
FROM
    total_payment_amounts_sum
ORDER BY
    year, month; -- Ensure final output is chronological
```

---